In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import autovarlambda
import time
from hyperopt import hp, rand, tpe, Trials, fmin
import pandas as pd
import yaml
import GPy
import GPyOpt
from numpy.random import seed

In [2]:
with open("varlam_gpyopt.yml", 'r') as stream:
    data_loaded = yaml.load(stream)

itype = data_loaded.get("itype")
if itype==1:
    print(" ==> algorithm type: GP")
if itype==2:
    print(" ==> algorithm type: random")

igrid = data_loaded.get("igrid") 
if igrid==0:
    print(" ==> grid type: continuous") 
if igrid==1:
    print(" ==> grid type: normal")

imin = data_loaded.get("imin") 
if imin==0:
    print(" ==> states included in loss function: ground + excited")
if imin==1:
    print(" ==> states included in loss function: excited")

ifun = data_loaded.get("ifun")
if ifun==0:
    print(" ==> loss functional: sum of weigthed relative errors")
if ifun==1:
    print(" ==> loss functional: sum of weigthed square relative errors")

maxevals = data_loaded.get("maxevals")
eps_up = data_loaded.get("eps_up")
eps_down = data_loaded.get("eps_down")
eps=eps_down/2.0
if eps_up > eps_down:
    eps=eps_up/2.0
nlamvar = data_loaded.get("nlamvar")
nener = data_loaded.get("nener")
igi = data_loaded.get("igi") 
if igi==0:
    print("using input weight")
    listweight = data_loaded.get("weight")
    NEMX = len(listweight)
    weight = np.array(listweight)
if igi==1:
    print("using statistical weight gi")
    NEMX = nener
    weight = np.full(nener,1.)
cfgs = data_loaded.get("cfgs")

 ==> algorithm type: GP
 ==> grid type: continuous
 ==> states included in loss function: ground + excited
 ==> loss functional: sum of weigthed square relative errors
using statistical weight gi


In [3]:
# Define useful functions
def atom_name(nzion):
    nzion=abs(nzion)
    atomname=['0','H','He','Li','Be','B','C','N','O','F','Ne','Na','Mg']
    for i in range(len(atomname)):
        if nzion==i:
            chatom = atomname[i]
    return chatom
def pot_type(nzion):
    if nzion < 0:
        chtype="STO"
    else:
        chtype="TFDA"
    return chtype

# Define function for writing iteration number in a human-readable-way
def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000
    return '%.f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

def error_relat(valexact,valcomp):
    if valexact == 0.0 or valcomp == 0.0:
        error=999
    error = abs((valexact-valcomp)/valexact)
    return error*100

# Loss function defined as weighted sum of energies relative errors
# def loss_wsumE(enere,enerc,neex,weight):
#     loss = 0.0
#     for i in range(neex):
#         diff = error_relat(enere[i],enerc[i])
#         loss = loss + weight[i]*diff
#     return loss
def loss_wsumE(enere,enerc,neex,weight):
    loss = 0.0
    if igi==1: # use statistical weight 
        weight=autovarlambda.compbck.gic
    for i in range(imin,neex): # consider ground energy and/or-only excited states
        diff = error_relat(enere[i],enerc[i])
        if ifun==1: # new functional 
            diff = diff*diff
        loss = loss + weight[i]*diff
    return loss

# Define function to be minimized
def var_lambda(x):
    minlam=0.1
    lam0 = x[0,:]
    nlam0 = len(lam0)
    for i in range(nlam0):
        if lam0[i] < minlam:
            lam0[i] = minlam
    autovarlambda.run_as(lam0,nlam0)
    neex = autovarlambda.exactbck.ne
    enere = autovarlambda.eneroutbck.enere
    enerc = autovarlambda.eneroutbck.enerc
    loss = loss_wsumE(enere,enerc,neex,weight)
    print(lam0,loss)
    return loss

# Determine file name string vector from fortran subroutine data
def def_filename():
    ncfg=autovarlambda.salgebbck.mxconf
    nzion=autovarlambda.salgebbck.nzion
    chatom=atom_name(nzion)
    chtype=pot_type(nzion)
    cfgs=str(ncfg)+"CFG"+chtype
    filename=chatom+cfgs+"_GP"+human_format(maxevals)
    return ncfg, filename

# Open .out file
def open_fout(filename,ncfg):
    fener = open(filename+".out","w")
    return fener

# Define and initialize variables
def init_var(NVMX,NEMX):
    lam0 = np.array(NVMX)
    nlam0 = NVMX
    enere = np.zeros(NEMX)
    enerc = np.zeros(NEMX)
    neex = int()
    return lam0, nlam0, enere, enerc, neex

def print_eresults():
    lam_best = myBopt.x_opt
    nlam0 = len(lam_best)
    autovarlambda.run_as(lam_best.reshape(-1,nlam0),nlam0)
    neex = autovarlambda.exactbck.ne
    enere = autovarlambda.eneroutbck.enere
    enerc = autovarlambda.eneroutbck.enerc
    loss_best = loss_wsumE(enere,enerc,neex,weight)
    #print("x*= {} f(x*)= {}".format(lam_best,loss_best))
    erroregr = error_relat(enere[0],enerc[0])
    print("\n===>>> print results in .out <<<===")
    print("\n Number of evaluations = {:10d}".format(maxevals),file=fener)
    print("                  Time = {:10.3f} minutes".format(total),file=fener)
    print("-"*80,file=fener)
    print(" Best results:",file=fener)
    print("-"*80,file=fener)
    print("\n  lambda = {} ".format(lam_best),file=fener)
    print("\n  Ground State Energy  = {:12.6f}".format(enerc[0]),
          "\n                  NIST = {:12.6f}".format(enere[0]),
          "\n                   Er% = {:12.4f} %".format(erroregr),file=fener)
    for i in range(1,neex):
        errorex = error_relat(enere[i],enerc[i])
        print("\n       {} Excit. Energy = {:12.6f}".format(i,enerc[i]),
              "\n                  NIST = {:12.6f}".format(enere[i]),
              "\n                   Er% = {:12.6f} %".format(errorex),file=fener)
    print("\n            Total loss = {:12.4f} %".format(loss_best),file=fener)
    fener.close()

In [4]:
# Initialize variables
lv = np.full(nlamvar, 1.)
NVMX=len(lv)
ntcfg = len(cfgs)
lam0, nlam0, enere, enerc, neex = init_var(NVMX,NEMX)

# Define the bounds of the function
bounds=[]
igrid=0
for i in range(NVMX):
    alam=str(i+1)
    if igrid==0:
        bounds.append({'name': 'lam'+alam, 
                       'type': 'continuous', 
                       'domain': (lv[i]-eps_down,lv[i]+eps_up)})

In [ ]:
# Begin loop in "configurations"
for i in range(ntcfg):
    icfg = cfgs[i]
    print("\n===>>> RUN {:3d} CFGs <<<===\n".format(icfg))
    cpdas = "cp das_" + str(icfg) + "CFG das"
    os.system(cpdas)
# Run initialization f2py subroutines
    autovarlambda.open_files()
    autovarlambda.inp_das()
    autovarlambda.inp_exact()

# if "ne" of exactvalues.dat is != than nener of .yaml
#  => "nener" is considered and ne is dumped
    dummyne = autovarlambda.exactbck.ne.copy()
    if nener != dummyne:
        print(" ************ nener != neex          ************")
        print(" ************ forcing: neex == nener ************\n")
        autovarlambda.exactbck.ne = nener

# Initialize files and necessary arrays
    ncfg, filename = def_filename()
    if ncfg != icfg:
        print(" >>>> configuration mismatch !!!! ")
        sys.exit()
    fener = open_fout(filename,ncfg)

# Creates three identical objects that we will later use to compare the optimization strategies 
    print("===>>> RUN GPyOpt <<<===")
    t0 = time.time()
    myBopt = GPyOpt.methods.BayesianOptimization(f=var_lambda,
                                                 domain=bounds,
                                                 model_type = 'GP',
                                                 acquisition_type='EI',  
                                                 normalize_Y = True,
                                                 acquisition_weight = 2)
# runs the optimization for the three methods
    myBopt.run_optimization(maxevals,verbosity=False)
    myBopt.save_evaluations(filename+".dat")
    myBopt.save_models(filename+".mod")
    myBopt.save_report(filename+".rep")
    t1 = time.time()
    total = (t1-t0)/60.

# run autostructure for best lambda found
    print_eresults()
    
os.system("rm CONFIG.DAT LEVELS TERMS olg oic ols tmp das OVRLAP")


===>>> RUN  32 CFGs <<<===

===>>> RUN GPyOpt <<<===
[1.93587068 0.53113239 0.39980812 0.62300345 1.2629869  1.77723628
 2.10580445 1.55551902 1.34123596 0.44526332 2.37282038 1.26685811
 0.94657992 0.30665979 2.08774149 1.8017662  0.21550114 2.3630349
 0.89507792 2.17293934] 94091.25330709406
[0.66692704 1.88027318 1.03150308 0.40914808 2.22104737 1.04039719
 2.06859563 0.12724218 0.23016222 2.48963826 1.77427683 0.42611293
 1.43381771 0.29737685 0.55016642 1.63249702 2.21246666 1.88848647
 2.00996413 0.58526971] 227688.67901528254
[0.79370232 1.17812169 2.06995648 1.25127636 2.4782504  1.01392547
 1.09330549 1.33764405 0.21058573 0.528538   2.37230271 2.36866911
 2.01595113 1.94114118 2.43888529 1.93848719 2.46908358 1.6452051
 0.55217619 2.01719866] 11577.976091674613
[1.83946869 1.57381387 1.1434563  0.77591981 2.40159387 1.77431954
 1.99941037 1.00993958 1.0737222  2.06924918 1.8529233  0.16907288
 0.51059054 1.74313566 1.97555804 2.46919133 2.22097843 2.18534409
 1.22260352 1.42

[0.93394632 1.46986635 2.13933709 1.60077896 2.5        1.34774187
 0.80212288 1.74693444 0.45815174 1.21555188 2.5        1.72317628
 1.42026557 1.97984756 2.5        2.34398871 2.21974344 2.48189282
 1.21368301 1.7496251 ] 1116.149289469722
[0.59759163 1.86879675 1.94274049 2.00224762 2.5        1.33908187
 1.20085249 1.38895423 1.33068628 1.15345792 2.5        1.822186
 0.63900275 1.71643092 2.5        1.23634147 2.1924699  1.74805667
 1.10588946 2.09313275] 17456.39646937616
[0.1 2.5 2.5 2.5 2.5 0.1 2.5 0.1 2.5 0.1 2.5 0.1 0.1 2.5 2.5 0.1 0.1 0.1
 2.5 2.5] 143959.11376465985
[0.52170799 1.82827073 2.02278774 2.13861027 2.5        1.26882556
 1.46199382 1.43329294 0.62063077 2.16198511 2.5        2.5
 0.5367466  2.40435301 2.5        0.1        0.1        2.31634493
 0.1        2.5       ] 13917.433921324317
[0.1        2.5        1.3118473  2.5        2.5        1.73464817
 1.56189881 2.5        2.00539009 2.5        2.5        2.5
 1.71367593 2.01789305 2.5        0.1        0.1  

[0.1        2.5        2.5        2.5        1.24490377 2.5
 0.1        0.1        2.5        2.5        0.1        0.1
 2.5        2.5        2.5        2.5        2.5        2.5
 0.1        0.1       ] 252952.14628177
[1.15381993 2.5        0.1        2.5        0.1        2.5
 0.1        2.5        0.92684907 2.5        2.5        2.5
 2.5        2.5        0.1        2.5        0.74585782 2.5
 0.1        0.15788597] 9126027.296323128
[0.1        2.5        2.5        2.09354139 1.82981138 2.5
 0.1        0.1        2.5        2.5        2.5        1.04505049
 1.95346022 0.81819093 0.94281399 0.1        0.1        0.1
 0.1        0.1       ] 213094.37670450012
[2.00864891 1.97222922 2.5        0.93038777 0.71974484 2.12399172
 0.1        0.1        0.77623505 0.46739334 2.5        1.65718361
 1.29510868 0.1        0.81836825 1.54026055 0.1        2.5
 1.92874483 0.50665459] 2754757.700400055
[0.1        2.5        2.5        2.5        2.5        2.5
 0.1        2.5        2.5      

[2.25150279 2.5        2.40641836 1.49010245 2.5        1.45297275
 2.5        0.71753538 0.31672612 2.5        2.5        0.1
 0.1        2.5        2.5        2.5        2.5        2.5
 2.5        1.67049965] 31921.915646047746
[0.1        2.5        2.5        2.5        2.5        2.5
 0.23674724 2.5        0.1        0.95884322 2.5        0.1
 0.1        2.5        2.5        0.1        0.1        0.1
 2.5        0.5827424 ] 559159.6922938556
[0.1        2.5        2.5        2.5        2.5        2.5
 0.1        1.26645969 2.5        2.5        0.1        0.1
 2.5        2.5        0.60551393 0.1        2.5        0.1
 0.1        0.1       ] 124695.41536229402
[0.1        0.1        2.5        2.5        2.5        2.5
 0.1        0.95341476 2.5        0.1        2.5        2.5
 0.1        2.5        0.1        2.5        2.5        0.1
 0.1        0.1       ] 137373.01744322907
[0.1        0.1        2.5        2.5        2.5        2.5
 2.18681525 0.7079317  2.5        2.5     

[0.1       0.9572154 2.5       2.5       2.5       2.5       0.1
 2.5       0.1       0.1       0.1       0.1       2.5       2.5
 2.5       2.5       2.5       0.1       0.1       0.1      ] 287507.5589533284
[1.95304575 1.34283593 1.23950934 0.66225152 1.03359199 2.08796848
 1.81487735 1.84685007 1.31949541 0.44001699 2.5        1.52562574
 0.79194663 1.11957539 1.55042259 1.91180041 0.46491871 2.40463497
 0.63266476 1.74025512] 23586.80597615913
[0.1        0.1        2.5        2.5        2.5        2.5
 2.5        0.83526536 0.1        0.1        2.5        2.5
 0.1        2.5        2.5        0.1        0.1        0.1
 0.79954762 0.1       ] 332828.3813157071
[0.1        2.5        2.5        2.5        2.5        1.65497207
 0.1        2.5        0.1        2.5        2.5        2.5
 2.5        1.70269565 2.5        0.1        0.1        1.33611465
 1.54522212 2.5       ] 67440.09272576157
[0.1 2.5 2.5 2.5 2.5 0.1 0.1 2.5 2.5 0.1 2.5 0.1 2.5 2.5 2.5 2.5 0.1 0.1
 0.1 0.1] 207610

...

[0.1        1.11749287 2.5        1.95562244 2.5        1.70061602
 1.88359603 0.2272634  0.1        2.5        2.5        1.54228429
 0.65619852 1.12238558 0.1        0.1        2.5        1.18255645
 0.95459593 0.1       ] 535004.9420232419
[0.1        0.49152968 2.5        2.5        2.5        0.1
 0.1        0.14654965 2.5        0.1        2.5        2.5
 2.5        2.5        2.5        2.5        2.5        0.1
 0.1        0.1       ] 222092.86020802276
[0.1        2.5        2.5        2.5        2.5        2.5
 0.1        2.41909562 0.1        0.1        0.1        0.1
 0.1        2.5        0.1        0.1        0.1        0.1
 0.1        0.1       ] 215127.36891275167
[2.5        2.5        2.18315619 2.5        2.5        2.5
 0.1        0.60205287 0.1        2.5        0.1        0.1
 0.1        2.5        2.5        2.5        2.5        2.5
 2.5        0.1       ] 171924.62179242977
[1.56525136 0.44880834 2.5        2.5        2.5        0.17204218
 1.15585025 1.6097750

[0.1        2.5        2.5        1.28390642 0.1        2.5
 2.5        2.5        0.1        0.1        0.3784325  0.1
 2.5        2.5        2.5        0.1        0.1        2.5
 0.1        2.5       ] 196152.21757162022
[0.1        2.5        2.5        1.7109474  1.16243456 2.5
 2.5        1.52273633 0.1        0.1        0.1        0.1
 0.1        2.5        2.5        1.11791617 2.5        2.5
 1.20250432 1.1517954 ] 110156.34045075311
[2.5        2.5        0.32264804 2.5        1.32640709 2.5
 0.1        0.1        0.1        0.1        0.51288172 2.5
 2.5        0.14345828 2.5        2.5        2.5        2.5
 2.5        0.1       ] 132652.9384420313
[0.1        2.5        2.5        2.5        2.5        2.5
 0.1        2.5        2.5        2.5        0.1        0.1
 2.5        2.5        2.5        0.1        0.1        2.5
 0.1        1.21700092] 85553.6808643116
[2.5        2.5        1.0294317  2.5        2.5        2.5
 0.1        0.1        0.1        2.5        1.1643

[0.1        0.57564922 2.5        2.5        2.5        0.1
 1.82434596 1.69329876 2.5        0.1        2.5        0.1
 0.1        2.5        2.5        2.5        0.1        0.1
 0.1        0.1       ] 123557.84642645966
[1.51136414 0.7249196  1.17909861 0.86782721 1.55391151 2.01401993
 2.5        2.14635077 1.47078685 0.1        2.5        1.32567091
 0.1        1.57402058 2.5        2.19327954 0.11239552 2.5
 1.0051688  1.28932582] 7695.864659055285
[0.1        0.1        2.5        2.5        2.5        0.1
 0.1        2.5        0.1        0.5953443  2.5        0.1
 2.5        0.83153938 0.1        0.1        0.1        0.1
 0.1        0.1       ] 392706.659736295
[0.1        2.5        2.5        2.5        0.56912317 2.5
 0.1        0.342718   2.5        0.1        0.1        0.1
 2.5        2.5        0.1        0.1        2.5        0.50878645
 0.1        0.1       ] 135465.46474505914
[0.1        2.5        2.5        2.5        0.94039749 2.5
 0.70872138 2.5        2.5    

[0.1        2.5        2.5        2.5        2.5        2.5
 0.1        2.0623885  0.1        2.5        0.1        0.1
 2.5        0.70733273 0.1        0.1        0.1        2.5
 2.5        0.1       ] 163270.5367575076
[0.1        2.5        1.40751215 2.5        0.72147647 2.5
 1.53439324 2.13451974 0.1        0.1        0.1        0.1
 2.5        0.1        2.5        2.5        0.1        2.5
 2.5        0.1       ] 147476.808274806
[0.1        1.28713915 2.5        2.5        2.5        2.5
 0.36203119 2.5        0.1        0.1        0.1        0.1
 0.1        2.5        2.5        2.5        2.5        2.5
 2.5        0.1       ] 165129.07199980508
[1.1436608  1.44445328 2.5        2.5        2.5        0.86243666
 1.19857685 0.7569268  0.78928601 1.89329388 2.5        2.5
 0.1        2.5        2.5        1.03499978 1.28106407 2.5
 1.1161458  2.5       ] 1941.8499219186388
[0.1        2.5        2.5        2.5        2.5        1.17446551
 1.05582454 2.5        0.1        2.5

[0.1        2.5        2.5        2.5        2.5        2.5
 0.39242109 2.5        2.5        2.5        2.5        2.5
 2.5        1.49018602 2.5        2.5        2.5        2.5
 2.5        2.5       ] 30496.194107327294
[1.92647179 2.5        2.5        2.5        2.5        1.26327908
 0.1        0.16749966 0.1        2.5        1.17866408 0.1
 1.05136287 2.5        0.38495781 0.1        0.1        0.1
 0.1        0.1       ] 591199.8969437012
[2.5        1.31717324 0.31541126 2.5        2.5        2.5
 0.1        2.14916865 0.1        0.1        0.1        0.1
 2.5        0.1        2.5        2.5        2.5        2.5
 2.5        0.1       ] 764488.9278848041
[0.1        2.5        2.5        1.78218454 2.5        1.54916391
 2.5        0.1        0.19635877 0.1        0.1        1.04381285
 0.1        2.5        2.5        2.5        2.5        1.12874681
 0.1        0.1       ] 48142.79099537923
[0.1        2.5        2.5        2.5        1.2871369  2.5
 1.09243284 0.92164878 

In [ ]:
print('lambda=',myBopt.x_opt,'f(lambda)=',myBopt.fx_opt)